In [18]:
import os
import pandas as pd 
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt


import os
import warnings
warnings.filterwarnings('ignore') 
from pandas_profiling import ProfileReport

In [19]:
%matplotlib inline

In [20]:
data = pd.read_csv('data/fff.csv',parse_dates=[['Crash Time','DATE']])#合并日期

# Exploring data

In [21]:
#df.head()

In [22]:
#profile = ProfileReport(data, title="Pandas Profiling Report")
#profile.to_file("plots/car_crashes_report.html")
#profile

In [24]:
#data = data.drop(['STATION','NAME','ID','Vehicle Action Prior to Crash',
                    #'Vehicle Travel Directions','X Cooordinate','Y Cooordinate'],axis=1)

# EDA & Feature 

In [25]:
for df in [data]:
    df['year'] = df['Crash Time_DATE'].dt.year
    df['month'] = df['Crash Time_DATE'].dt.month
    df['day'] = df['Crash Time_DATE'].dt.day
    df['hour'] = df['Crash Time_DATE'].dt.hour

    df['hour_category'] = np.nan
    df['season'] = np.nan
    df['week'] = np.nan
    
    df.loc[(df.hour <= 9)&(df.hour >= 7), 'hour_category'] = "rush"
    df.loc[(df.hour <= 19)&(df.hour >= 16), 'hour_category'] = "rush"
    df.loc[(df.hour < 16)&(df.hour > 9), 'hour_category'] = "day"
    df.loc[(df.hour < 7)|(df.hour > 19), 'hour_category'] = "night"
    
    df.loc[(df.month <= 3)&(df.month >= 1), 'season'] = 1
    df.loc[(df.month <= 6)&(df.month >= 4), 'season'] = 2
    df.loc[(df.month <=9)&(df.month >= 7), 'season'] = 3
    df.loc[(df.month <=12)&(df.month >=10), 'season'] = 4
    
    df.loc[(df.day <= 7)&(df.day >= 1), 'week'] = 1
    df.loc[(df.day <= 14)&(df.day >= 8), 'week'] = 2
    df.loc[(df.day <=21)&(df.day >= 15), 'week'] = 3
    df.loc[(df.day >=22), 'week'] = 4

# Convert categorical variable into dummy variables

In [26]:
#Severity = pd.get_dummies(data['Crash Severity'], prefix='Severity', prefix_sep='_')

#Injury_Reported = pd.get_dummies(data['Maximum Injury Severity Reported'], prefix='Injury_Reported', prefix_sep='_')

#Manner = pd.get_dummies(data['Manner of Collision'], prefix='Manner', prefix_sep='_')

Surface_Condition = pd.get_dummies(data['Road Surface Condition'], prefix='Condition', prefix_sep='_')

Ambient_Light = pd.get_dummies(data['Ambient Light'], prefix='Light', prefix_sep='_')

Weather_Condition = pd.get_dummies(data['Weather Condition'], prefix='Weather', prefix_sep='_')

#Ice = pd.get_dummies(data['Ice'], prefix='Ice', prefix_sep='_')

Hour = pd.get_dummies(data['hour_category'], prefix='Hour', prefix_sep='_')

Season = pd.get_dummies(data['season'], prefix='season', prefix_sep='_')

Week = pd.get_dummies(data['week'], prefix='week', prefix_sep='_')


In [27]:
Data_Master = pd.concat([data,
                          Surface_Condition,
                          Ambient_Light,
                          Weather_Condition,
                          Hour,
                          Season,
                          Week
                         ], axis=1)

In [28]:
#data = Data_Master.drop(['Crash Severity','Maximum Injury Severity Reported',
                                 # 'Manner of Collision','Road Surface Condition',
                                  #'Ambient Light','Weather Condition','Ice','hour_category','Crash Time_DATE',
                                  #'Number of Vehicles','Number of Vehicles','Total Fatal Injuries','label',
                                  #'Total Nonfatal Injuries' ], axis = 1)

In [29]:
#data = Data_Master.drop(['Crash Severity','Maximum Injury Severity Reported',
                                  #'Manner of Collision','Road Surface Condition',
                                 # 'Ambient Light','Weather Condition','Ice','hour_category','Crash Time_DATE'], axis = 1)

In [30]:
data.columns

Index(['Crash Time_DATE', 'Unnamed: 0', 'ID', 'Month', 'Day of week',
       'Road Surface Condition', 'Ambient Light', 'Weather Condition', 'PRCP',
       'SNOW', 'TMAX', 'TMIN', 'lon', 'lat', 'label', 'year', 'month', 'day',
       'hour', 'hour_category', 'season', 'week'],
      dtype='object')

# Data partition

In [31]:
split_size = 0.7
train = data[0:int(split_size*len(data))]
test = data[int(split_size*len(data))::]

In [32]:
X_train = train.drop(['lon','lat'], axis=1)
Y_train = pd.concat([train['lon'],train['lat']], axis=1)

X_test = test.drop(['lon','lat'], axis=1)
Y_test = pd.concat([test['lon'],test['lat']], axis=1)

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15534 entries, 0 to 15533
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Crash Time_DATE         15534 non-null  datetime64[ns]
 1   Unnamed: 0              15534 non-null  int64         
 2   ID                      15534 non-null  int64         
 3   Month                   15534 non-null  object        
 4   Day of week             15534 non-null  object        
 5   Road Surface Condition  15534 non-null  object        
 6   Ambient Light           15534 non-null  object        
 7   Weather Condition       15534 non-null  object        
 8   PRCP                    15534 non-null  float64       
 9   SNOW                    15534 non-null  float64       
 10  TMAX                    15534 non-null  int64         
 11  TMIN                    15534 non-null  int64         
 12  lon                     15534 non-null  float6

# regression model

In [34]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor
def xgb_autotune(X,y):
    param = {
            'objective':'reg:squarederror',
            'n_estimators':1000,
            'learning_rate':0.05,
            'gamma':0,
            'max_depth':5,
            'min_child_weight':1,
            'colsample_bytree':0.5,
            'subsample': 0.8, 
            'verbose':1,
            'reg_alpha':0,
            'tree_method': 'gpu_hist'
    }


    param_test1 = {
     'max_depth':[7,13,15],
     'reg_alpha':[0.1,1,10,100],
     'learning_rate':[0.05,0.1],
    }

    gsc = GridSearchCV(estimator=xgb.XGBRegressor(objective=param['objective'],
                              n_estimators=param['n_estimators'], learning_rate = param['learning_rate'], gamma=param['gamma'],
                              colsample_bytree=param['colsample_bytree'], subsample=param['subsample'], reg_alpha=param['reg_alpha']),param_grid=param_test1,
                        cv=5, 
                        scoring='neg_mean_squared_error', 
                        verbose=0,
                        n_jobs=-1)
    gsearch = MultiOutputRegressor(gsc).fit(X,y)
    return gsearch
gsearch = xgb_autotune(X_train,Y_train)

ValueError: DataFrame.dtypes for data must be int, float, bool or category.  When
categorical type is supplied, DMatrix parameter `enable_categorical` must
be set to `True`. Invalid columns:Crash Time_DATE, Month, Day of week, Road Surface Condition, Ambient Light, Weather Condition, hour_category

In [35]:
print(gsearch.estimators_[0].best_params_)
print(gsearch.estimators_[1].best_params_)

NameError: name 'gsearch' is not defined

In [36]:
param = {
            'objective':'reg:squarederror',
            'n_estimators':1000,
            'learning_rate':0.05,
            'gamma':0,
            'min_child_weight':1,
            'colsample_bytree':0.5,
            'subsample': 0.8, 
            'verbose':1,
            'tree_method': 'gpu_hist',
            'max_depth': 7, 
            'reg_alpha': 100 
    }

In [41]:
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
from tqdm import tqdm
multi_xgb = MultiOutputRegressor(XGBRegressor(param_grid=param))
multi_xgb = multi_xgb.fit(X_train,Y_train)

ValueError: DataFrame.dtypes for data must be int, float, bool or category.  When
categorical type is supplied, DMatrix parameter `enable_categorical` must
be set to `True`. Invalid columns:Crash Time_DATE, Month, Day of week, Road Surface Condition, Ambient Light, Weather Condition, hour_category

In [39]:
import joblib
joblib.dump(multi_xgb, 'models/multi_xgb.model')

['models/multi_xgb.model']

In [40]:
def haversine_dist(lat1,lng1,lat2,lng2):
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    radius = 6371  # Earth's radius taken from google
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat/2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng/2) ** 2
    h = 2 * radius * np.arcsin(np.sqrt(d))
    return h

# Prediction

In [38]:
results = multi_xgb.predict(X_test)
np.savetxt("data/result.csv", results, delimiter=',')
results

NotFittedError: This MultiOutputRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

# Predictions VS Values

In [41]:
Longtitutes_pred = [ result[0] for result in results]
Latitutes_pred = [ result[1] for result in results]
Longtitutes_gt = Y_test['Longtitute'].values
Latitutes_gt = Y_test['Latitute'].values
dis=haversine_dist(Latitutes_pred,Longtitutes_pred,Latitutes_gt,Longtitutes_gt)

In [42]:
Longtitutes_gt

array([41.92416457, 41.89912097, 41.95466142, ..., 41.91931   ,
       41.93628409, 41.90423299])

In [43]:
np.mean(haversine_dist(Latitutes_pred,Longtitutes_pred,Latitutes_gt,Longtitutes_gt))

3.592262979461295

In [44]:
filename = open('data/attributes.txt', 'w')  
for value in data.columns:  
    filename.write(str(value)+'\n') 
filename.close() 

In [45]:
disd=pd.DataFrame(dis)

In [46]:
disd.describe()

,0
count,5454.000000
mean,3.592263
std,1.896259
min,0.017813
25%,2.093115
50%,3.622916
75%,4.862171
max,10.560092


In [47]:
#0.006365004

In [48]:
Latitutes_gt

array([-71.35104162, -71.35505451, -71.28863588, ..., -71.29916643,
       -71.28608551, -71.34958247])

In [49]:
help(haversine_dist)

Help on function haversine_dist in module __main__:

haversine_dist(lat1, lng1, lat2, lng2)



In [53]:
Location=pd.DataFrame(results)
Location

,0,1
0,41.918285,-71.348068
1,41.926617,-71.326752
2,41.931118,-71.326775
3,41.932087,-71.318451
4,41.928341,-71.292877
...,...,...
5449,41.927746,-71.310173
5450,41.930061,-71.313484
5451,41.932064,-71.289719
5452,41.936432,-71.306702


In [56]:
Location.columns =['longtitute', 'latitute']
Location

,longtitute,latitute
0,41.918285,-71.348068
1,41.926617,-71.326752
2,41.931118,-71.326775
3,41.932087,-71.318451
4,41.928341,-71.292877
...,...,...
5449,41.927746,-71.310173
5450,41.930061,-71.313484
5451,41.932064,-71.289719
5452,41.936432,-71.306702


In [63]:
from folium import plugins
import numpy as np 
import pandas as pd 
import folium
import webbrowser
from folium.plugins import HeatMap

In [64]:
import statistics
meanLong = statistics.mean(Location['longtitute'])
meanLat = statistics.mean(Location['latitute'])

In [65]:
m = folium.Map(location=[meanLat, meanLong], zoom_start = 14.5)

In [66]:
LocationM = Location.values

In [67]:
m.add_child(plugins.HeatMap(LocationM, radius=13))